# Bayesian analysis of emotion-mediated memory



In this notebook I will conduct (somewhat) speedy Bayesian mixed-effects logistic regression to test how word-level features predict memory, with subject-level random effects. 


Notes
___


- Bambi needs up-to-date xarray (for 'unify_chunks' method), while the data I pickled used xarray 0.13.0 


- Bambi can't multi-process sampling across multiple chains if you import joblib, so need to make sure to only load in csv data. 

In [ ]:
import arviz as az
import bambi as bmb
import argparse
from os.path import join
import pandas as pd
from pandas.api.types import CategoricalDtype
import seaborn as sns 
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from Bayesian_model_utils import run_model, plot_res, print_latex_table, plot_predictions

## How do arousal and valence predict recall performance?

In [ ]:
# First, let's build our model and plot the priors to see if they're sensible 

# Old dataset of pts with Hipp or Amy electrodes 
behav_df = pd.read_csv('/home1/salman.qasim/Salman_Project/FR_Emotion/events_all.csv', low_memory=False)

# Get only encoding events
behav_df = behav_df[behav_df.type=='WORD']

# Center valence using mean of whole database to capture relative polarity
behav_df.valence = behav_df.valence - 0.5

behav_df = behav_df[['recalled', 'arousal', 'valence', 'subject']]

y = 'recalled'
X = ['arousal', 'valence'] 
Intx = ['arousal:valence']
rand_effect = ['subject']

# Drop nan data
behav_df = behav_df.dropna(subset=X)

label = (f"{y}" + "_{}"*len(X)).format(*X) + '_allFRpts'

# Set some terms 
rand_term = [f'(1|{x})' for x in rand_effect]
formula = f'{y} ~ 1+'+'+'.join(rand_term)+'+'+'+'.join(X)+'+'+'+'.join(Intx)
model_fam = 'bernoulli'
priors=None
categorical=None

# construct the model 
model = bmb.Model(formula=formula, 
              data=behav_df[rand_effect + [y] + X],
             family=model_fam,
             priors=priors,
             categorical=categorical)

model.build()

model.plot_priors()

In [ ]:
# Make the graphical model: 

model.graph()

Our model formulation looks sensible. Let's do it.

In [ ]:
# Model 1: Effect of arousal and continuous, linear valence on recall 

behav_df = pd.read_csv('/home1/salman.qasim/Salman_Project/FR_Emotion/events_all.csv', low_memory=False)

# Get only encoding events
behav_df = behav_df[behav_df.type=='WORD']

# Center valence using mean of whole database to capture relative polarity
behav_df.valence = behav_df.valence - 0.5

behav_df = behav_df[['recalled', 'arousal', 'valence', 'subject']]

y = 'recalled'
X = ['arousal', 'valence'] 
Intx = ['arousal:valence']
rand_effect = ['subject']

# Drop nan data
behav_df = behav_df.dropna(subset=X)

label = (f"{y}" + "_{}"*len(X)).format(*X) + '_allFRpts'

# Run the model 
run_model(behav_df, y, X, Intx, rand_effect, 
          chains=4, cores=4, tune=1000, draws=1000, 
          label = label, rand_slopes=False)

In [ ]:
# Model 2: Effect of arousal and binned valence on recall 

behav_df = pd.read_csv('/home1/salman.qasim/Salman_Project/FR_Emotion/events_all.csv', low_memory=False)

behav_df = behav_df.drop(columns=['Unnamed: 0'])

# Get only encoding events
behav_df = behav_df[behav_df.type=='WORD']

behav_df = behav_df[['recalled', 'arousal', 'CV', 'subject']]

y = 'recalled'
X = ['arousal', 'CV'] 
Intx = ['arousal:CV']
rand_effect = ['subject']
# Drop nan data
behav_df = behav_df.dropna(subset=X)

label = (f"{y}" + "_{}"*len(X)).format(*X) + '_allFRpts'

# Run the model 
run_model(behav_df, y, X, Intx, rand_effect, categorical=categorical, 
          chains=4, cores=4, tune=1000, draws=1000, 
          label = label, rand_slopes=False)

In [ ]:
# Model 3: Effect of arousal and squared valence on recall 

behav_df = pd.read_csv('/home1/salman.qasim/Salman_Project/FR_Emotion/events_all.csv', low_memory=False)

behav_df = behav_df.drop(columns=['Unnamed: 0'])

# Get only encoding events
behav_df = behav_df[behav_df.type=='WORD']

# Center valence using mean of whole database to capture relative polarity
behav_df.valence = behav_df.valence - 0.5

behav_df['valence_squared'] = behav_df.valence**2

behav_df = behav_df[['recalled', 'arousal', 'valence_squared', 'subject']]

y = 'recalled'
X = ['arousal', 'valence_squared'] 
Intx = ['arousal:valence_squared']
rand_effect = ['subject']
categorical =['CV'] 
# Drop nan data
behav_df = behav_df.dropna(subset=X)

label = (f"{y}" + "_{}"*len(X)).format(*X) + '_allFRpts'

# Run the model 
run_model(behav_df, y, X, Intx, rand_effect, categorical=categorical, 
          chains=4, cores=4, tune=1000, draws=1000, 
          label = label, rand_slopes=False)

Let's now do model comparison using leave-one-out cross-validation and determine which formulation of valence is best for prediction. 

Source:
https://arviz-devs.github.io/arviz/api/generated/arviz.compare.html

https://arviz-devs.github.io/arviz/api/generated/arviz.plot_compare.html

https://bambinos.github.io/bambi/main/notebooks/model_comparison.html?highlight=waic


In [ ]:
# Model comparison: use PSI-LOO

# Load each model
save_dir = '/home1/salman.qasim/Salman_Project/FR_Emotion/BayesModels'
linear_valence = az.from_netcdf(f'{save_dir}/recalled_arousal_valence_allFRpts_model')
valence_squared = az.from_netcdf(f'{save_dir}/recalled_arousal_valence_squared_allFRpts_model')
binned_valence = az.from_netcdf(f'{save_dir}/recalled_arousal_CV_allFRpts_model')

models = {"linear_valence": linear_valence, 
          "valence_squared": valence_squared,
         "binned_valence": binned_valence}

df_compare = az.compare(models, ic='loo')
plot_df = df_compare.reset_index().rename(columns={'index':'model'})[['model', 'loo']]

c_string = '|c'*plot_df.shape[1] + '|'
print(plot_df.to_latex(index=False, 
                                        column_format=c_string).replace("\\\n", "\\ \hline\n"))  
az.plot_compare(df_compare, insample_dev=True)
plt.savefig(f'{save_dir}/behav_valence_comparison.pdf')

In [ ]:
az.plot_compare(df_compare, insample_dev=True)
plt.savefig(f'{save_dir}/arousal_valence_comparison.pdf')

## How do arousal and valence interact with direct-brain stimulation ?

In [ ]:
# Model 1: Effect of stimulation to Hipp only on recall as a function of arousal and valence: 

stim_behav_df = pd.read_csv('/home1/salman.qasim/Salman_Project/FR_Emotion/all_stim_ev.csv')

# Get only encoding events
stim_behav_df = stim_behav_df[stim_behav_df.type=='WORD']

# Get only hippocampal stim:
stim_behav_df = stim_behav_df[(stim_behav_df.stim_reg=='Hipp')]
 
# Center valence using mean of whole database to capture relative polarity
stim_behav_df['valence'] = stim_behav_df['valence'] - 0.5

cat_type = CategoricalDtype(categories=['right', 'left'], ordered=True)
stim_behav_df.stim_hemi = stim_behav_df.stim_hemi.astype(cat_type)

stim_behav_df = stim_behav_df[['recalled', 'valence', 'arousal', 'subject', 'is_stim', 'stim_hemi']]

y = 'recalled'
X = ['is_stim', 'arousal', 'valence', 'stim_hemi'] 
Intx = ['is_stim:arousal', 'is_stim:valence']

rand_effect = ['subject']
categorical = ['is_stim', 'stim_hemi']
label = (f"{y}" + "_{}"*len(X)).format(*X) + '_HippOnly' 

# Drop nan data
stim_behav_df = stim_behav_df.dropna(subset=X)

run_model(stim_behav_df, y, X, Intx, rand_effect, chains=4, cores=4, tune=2000, draws=2000, target_accept=0.95,
          rand_slopes=False, categorical=categorical, label=label)

In [ ]:
# Model 2: Effect of stimulation to MTL only on recall as a function of arousal and valence: 

stim_behav_df = pd.read_csv('/home1/salman.qasim/Salman_Project/FR_Emotion/all_stim_ev.csv')

# Get only encoding events
stim_behav_df = stim_behav_df[stim_behav_df.type=='WORD']

# Get only hippocampal stim:
stim_behav_df = stim_behav_df[(stim_behav_df.stim_reg=='MTL')]
 
# Center valence using mean of whole database to capture relative polarity
stim_behav_df['valence'] = stim_behav_df['valence'] - 0.5

cat_type = CategoricalDtype(categories=['right', 'left'], ordered=True)
stim_behav_df.stim_hemi = stim_behav_df.stim_hemi.astype(cat_type)

stim_behav_df = stim_behav_df[['recalled', 'valence', 'arousal', 'subject', 'is_stim', 'stim_hemi']]

y = 'recalled'
X = ['is_stim', 'arousal', 'valence', 'stim_hemi'] 
Intx = ['is_stim:arousal', 'is_stim:valence']

rand_effect = ['subject']
categorical = ['is_stim', 'stim_hemi']
label = (f"{y}" + "_{}"*len(X)).format(*X) + '_MTLOnly' 

# Drop nan data
stim_behav_df = stim_behav_df.dropna(subset=X)

run_model(stim_behav_df, y, X, Intx, rand_effect, chains=4, cores=4, tune=2000, draws=2000, target_accept=0.95,
          rand_slopes=False, categorical=categorical, label=label)

## How do arousal and valence interact with direct-brain stimulation ?

In [ ]:
# Model 1: Account for both BDI and BAI scores

# First, load the Beck Scores: 
BeckScores = pd.read_csv('/home1/salman.qasim/Salman_Project/FR_Emotion/Beck_Scores.csv')
BeckScores.rename(columns={'Subject Code':'subject'}, inplace=True)

behav_df = pd.read_csv('/home1/salman.qasim/Salman_Project/FR_Emotion/events_all_patients_revision.csv', memory_map=True)

# Get only encoding events
behav_df = behav_df[behav_df.type=='WORD']

# add in the Beck Scores 
behav_df = behav_df.merge(BeckScores, on='subject')

behav_df['valence'] = behav_df['valence'] - 0.5

behav_df = behav_df[['recalled', 'BDI', 'BAI', 'subject']]

y = 'recalled'
X = ['BDI', 'BAI']
Intx= ['BDI:BAI']

rand_effect = ['subject']

# Drop nan data
behav_df = behav_df.dropna(subset=X)

label = (f"{y}" + "_{}"*len(X)).format(*X) + '_allFRpts'

# Run the model 
run_model(behav_df, y, X, Intx, rand_effect, chains=4, cores=4, tune=1000, draws=1000, 
          label = label, rand_slopes=False)

In [ ]:
# Model 2: Account for just BDI and interaction with arousal

# First, load the Beck Scores: 
BeckScores = pd.read_csv('/home1/salman.qasim/Salman_Project/FR_Emotion/Beck_Scores.csv')
BeckScores.rename(columns={'Subject Code':'subject'}, inplace=True)

behav_df = pd.read_csv('/home1/salman.qasim/Salman_Project/FR_Emotion/events_all_patients_revision.csv', memory_map=True)

# Get only encoding events
behav_df = behav_df[behav_df.type=='WORD']

# add in the Beck Scores 
behav_df = behav_df.merge(BeckScores, on='subject')

# make squared valence
behav_df['valence'] = behav_df['valence'] - 0.5

behav_df = behav_df[['recalled', 'subject', 'BDI', 'arousal']]

y = 'recalled'
X = ['BDI', 'arousal']
Intx= ['BDI:arousal']

rand_effect = ['subject']

# Drop nan data
behav_df = behav_df.dropna(subset=X)

label = (f"{y}" + "_{}"*len(X)).format(*X) + '_allFRpts'

# Run the model 
run_model(behav_df, y, X, Intx, rand_effect, chains=4, cores=4, tune=1000, draws=1000, 
          label = label, rand_slopes=False)

In [ ]:
# Model 2: Account for just BDI and interaction with valence

# First, load the Beck Scores: 
BeckScores = pd.read_csv('/home1/salman.qasim/Salman_Project/FR_Emotion/Beck_Scores.csv')
BeckScores.rename(columns={'Subject Code':'subject'}, inplace=True)

behav_df = pd.read_csv('/home1/salman.qasim/Salman_Project/FR_Emotion/events_all_patients_revision.csv', memory_map=True)

# Get only encoding events
behav_df = behav_df[behav_df.type=='WORD']

# add in the Beck Scores 
behav_df = behav_df.merge(BeckScores, on='subject')

# make squared valence
behav_df['valence'] = (behav_df.valence - 0.5)

behav_df = behav_df[['recalled', 'subject', 'BDI', 'valence']]

y = 'recalled'
X = ['BDI', 'valence']
Intx= ['BDI:valence']

rand_effect = ['subject']

# Drop nan data
behav_df = behav_df.dropna(subset=X)

label = (f"{y}" + "_{}"*len(X)).format(*X) + '_allFRpts'

# Run the model 
run_model(behav_df, y, X, Intx, rand_effect, chains=4, cores=4, tune=1000, draws=1000, 
          label = label, rand_slopes=False)

Is BDI more predictive of memory when taking word arousal or word valence into account?

In [ ]:
# Model comparison: use PSI-LOO

# Load each model
save_dir = '/home1/salman.qasim/Salman_Project/FR_Emotion/BayesModels'
BDI_arousal = az.from_netcdf(f'{save_dir}/recalled_BDI_arousal_allFRpts_model')
BDI_valence = az.from_netcdf(f'{save_dir}/recalled_BDI_valence_squared_allFRpts_model')

models = {"BDI_arousal": BDI_arousal, 
          "BDI_valence": BDI_valence}

df_compare = az.compare(models, ic='loo')
plot_df = df_compare.reset_index().rename(columns={'index':'model'})[['model', 'loo']]

c_string = '|c'*plot_df.shape[1] + '|'
print(plot_df.to_latex(index=False, 
                                        column_format=c_string).replace("\\\n", "\\ \hline\n"))  
az.plot_compare(df_compare, insample_dev=True)
plt.savefig(f'{save_dir}/BDI_arousal_valence_comparison.pdf')